In [1]:
import pickle
from nltk.sentiment.util import *
from nltk.corpus import stopwords
from nltk.corpus import TwitterCorpusReader
from nltk.tokenize import TweetTokenizer


In [2]:
#tweets: list of strings ; lemmatizer: nltk Lemmatizer ; stemmer : nltk Stemmer
#remove stopword and tokenize strings. lemmatize/stemming if lemmatizer/stemmer not None
def preprocessString(tweets, lemmatizer, stemmer):
    
    #stopword
    tweets = [w for w in tweets if w.lower() not in stopwords.words('english')]
    #tokenize
    tokenizer = TweetTokenizer()
    tweets = [tokenizer.tokenize(tweet) for tweet in tweets]
    #lower
    for i in range(len(tweets)):
        tweets[i] = [t.lower() for t in tweets[i]]
    
    if lemmatizer != None:
        for i in range(len(tweets)):
            #lemmatization 
            tweets[i] = [lemmatizer.lemmatize(t) for t in tweets[i]]
    if stemmer != None:
        for i in range(len(tweets)):
            #stemming 
            tweets[i] = [stemmer.stem(t) for t in tweets[i]]
    
    #Collocations, Bigrams, Trigrams
    #Chunking
            
    return tweets

#tweets: list of objects, tag: tag in string 
def add_tags(tweets, tag):
    return [[tweet, tag] for tweet in tweets]

def identity(x):
    return x

In [3]:
#StreamListener
from secret import *
import tweepy
import pandas as pd
import json


auth = tweepy.OAuthHandler(C_KEY, C_SECRET)  
auth.set_access_token(A_TOKEN, A_TOKEN_SECRET)  
api = tweepy.API(auth)


#from tweepy API
class TBStreamListener(tweepy.StreamListener):

    def __init__(self,api=None):
        super().__init__(api)
        self.tweets = []
        self.numTweet = 20


    def on_data(self, data):
        if self.numTweet > 0:
            data = json.loads(data)
            self.tweets.append(data['text'])
            self.numTweet -= 1
        else:
            return False

    def on_status(self, status):
        print(status.text)

    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

In [4]:
#grab tweet
tbStreamListener = TBStreamListener()
stream = tweepy.Stream(auth = api.auth, listener=tbStreamListener)

#not both sample and filter
#stream.sample(True, languages = ['en'])
stream.filter(track=['twitter'], languages = ['en']) #, async=True)

In [5]:
#need to wait for a while
tbStreamListener.tweets

['RT @chelseynvana: happy ass potato https://t.co/5YYX58rSdO',
 'RT @NoahCRothman: WTF. "Some Coral Springs police were stunned and upset that the four original Broward County Sheriff\'s deputies who were…',
 "Here's the official 💯➕ club of 13 strong from the testing results at Sunday's #TheOpening regional in Miami https://t.co/ETQb0yDb0K",
 'RT @snmrrw: Facebook is at CPAC and they have a VR shooting game https://t.co/wmV23jezpN',
 'Starting to post some #STL photography from my archives.  #photography #photoshare #STLphotos  check out… https://t.co/UGeALwJxMX',
 'RT @OnlyHipHopFacts: 19 years ago today, Eminem released The Slim Shady LP. https://t.co/bs0qjbMrS4',
 "RT @connor_hannigan: when you're full on chips &amp; salsa and your food arrives https://t.co/fa76MET61R",
 'Hey @FedEx - please join us and end your relationship with the @NRA. #BoycottNRA #NeverAgain #ArmMeWith https://t.co/sZEukedWI4',
 "Can't forget this day when I enjoyed this gorgeous beauty while u sitting in front

In [6]:
#load data
#from nltk.corpus import twitter_samples
#tweet = twitter_samples.strings(fileids = 'negative_tweets.json')

In [9]:
tweet = tbStreamListener.tweets

In [10]:
tweet[2]

"Here's the official 💯➕ club of 13 strong from the testing results at Sunday's #TheOpening regional in Miami https://t.co/ETQb0yDb0K"

In [11]:

#lemmatization
wnl = nltk.WordNetLemmatizer()
#stemming
pstemmer = nltk.PorterStemmer()
#process tweets
test_data = preprocessString(tweet, wnl, pstemmer)


In [12]:
#vectorizer
vectorizer = pickle.load(open("tfidf_veczr.pkl","rb"))
testX = vectorizer.transform(test_data)

In [13]:
#classifier
classifier = pickle.load(open("SVM_clf.pkl","rb"))
classifier.predict(testX)

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1])